### **Step 1: Define the Environment**

The environment represents the solar power system, including the solar panel, battery, and their interactions, with **time of the day** factored in.

1. **State (What the agent knows at any moment):**
   The state includes all the information the agent uses to make decisions:

   - **Current solar power output** (e.g., in kW).
   - **Current battery charge level** (e.g., percentage or kWh).
   - **Current time of the day** (e.g., hour, or normalized value between 0–1).
   - **Optional:** Current power demand/output (if the system balances demand directly).

   Example state:  
   `[solar_power=2.5 kW, battery_level=70%, time=12:00]`

2. **Actions (What the agent can do):**

   - **Charge the battery** (store energy in the battery).
   - **Discharge the battery** (release energy from the battery).
   - **Do nothing** (leave the system unchanged).

3. **Reward (How the agent learns):**
   Assign rewards to encourage efficient energy management:
   - **Positive reward**:
     - Charging during times of high solar power (e.g., midday).
     - Discharging to meet constant output when solar is low (e.g., at night).
     - Avoiding overcharge or deep discharge of the battery.
   - **Negative reward (penalty)**:
     - Overcharging the battery (exceeding max capacity).
     - Draining the battery to zero (causing inefficiency or downtime).
     - Wasting solar power by not charging when solar power is abundant.

---


In [5]:
import pandas as pd

df = pd.read_csv("smartBMS\smartBMS\Plant_2_Generation_Data.csv")
df

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\gwiku\AppData\Local\Temp\ipykernel_128884\4090026835.py:3: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv("smartBMS\smartBMS\Plant_2_Generation_Data.csv")


,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15 00:00:00,4136001,4UPUqMRk7TRMgml,0.0,0.0,9425.000000,2.429011e+06
1,2020-05-15 00:00:00,4136001,81aHJ1q11NBPMrL,0.0,0.0,0.000000,1.215279e+09
2,2020-05-15 00:00:00,4136001,9kRcWv60rDACzjR,0.0,0.0,3075.333333,2.247720e+09
3,2020-05-15 00:00:00,4136001,Et9kgGMDl729KT4,0.0,0.0,269.933333,1.704250e+06
4,2020-05-15 00:00:00,4136001,IQ2d7wF4YD8zU1Q,0.0,0.0,3177.000000,1.994153e+07
...,...,...,...,...,...,...,...
67693,2020-06-17 23:45:00,4136001,q49J1IKaHRwDQnt,0.0,0.0,4157.000000,5.207580e+05
67694,2020-06-17 23:45:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,3931.000000,1.211314e+08
67695,2020-06-17 23:45:00,4136001,vOuJvMaM2sgwLmb,0.0,0.0,4322.000000,2.427691e+06
67696,2020-06-17 23:45:00,4136001,xMbIugepa2P7lBB,0.0,0.0,4218.000000,1.068964e+08


In [31]:
import pandas as pd

def normalize_time_from_datetime(datetime_value):
    # Extract the time part
    total_seconds = (
        datetime_value.hour * 3600
        + datetime_value.minute * 60
        + datetime_value.second
    )
    # Normalize the time
    normalized_time = total_seconds / 86400  # Total seconds in a day
    return normalized_time

# Example usage
datetime_value = pd.to_datetime("2024-12-21 00:00:1")
normalized = normalize_time_from_datetime(datetime_value)
print(f"Normalized time: {normalized}")


Normalized time: 1.1574074074074073e-05


In [7]:
df_cleaned = df[["DATE_TIME","DC_POWER"]]
# # lable encode the source key column
# df_cleaned["SOURCE_KEY"] = df_cleaned["SOURCE_KEY"].astype('category').cat.codes
df_cleaned

,DATE_TIME,DC_POWER
0,2020-05-15 00:00:00,0.0
1,2020-05-15 00:00:00,0.0
2,2020-05-15 00:00:00,0.0
3,2020-05-15 00:00:00,0.0
4,2020-05-15 00:00:00,0.0
...,...,...
67693,2020-06-17 23:45:00,0.0
67694,2020-06-17 23:45:00,0.0
67695,2020-06-17 23:45:00,0.0
67696,2020-06-17 23:45:00,0.0


In [8]:
# group  df_cleaned by SOURCE_KEY and DATE_TIME and sum the DC_POWER
df_cleaned_grouped = df_cleaned.groupby(["DATE_TIME"]).sum()
df_cleaned_grouped

,DC_POWER
DATE_TIME,
2020-05-15 00:00:00,0.0
2020-05-15 00:15:00,0.0
2020-05-15 00:30:00,0.0
2020-05-15 00:45:00,0.0
2020-05-15 01:00:00,0.0
...,...
2020-06-17 22:45:00,0.0
2020-06-17 23:00:00,0.0
2020-06-17 23:15:00,0.0


In [9]:
# make the DATE_TIME column just a column and not an index 
df_cleaned_grouped.reset_index(inplace=True)
df_cleaned_grouped

,DATE_TIME,DC_POWER
0,2020-05-15 00:00:00,0.0
1,2020-05-15 00:15:00,0.0
2,2020-05-15 00:30:00,0.0
3,2020-05-15 00:45:00,0.0
4,2020-05-15 01:00:00,0.0
...,...,...
3254,2020-06-17 22:45:00,0.0
3255,2020-06-17 23:00:00,0.0
3256,2020-06-17 23:15:00,0.0
3257,2020-06-17 23:30:00,0.0


In [10]:
# remove the date from the DATE_TIME column and keep only the time , and create a new column for the date
df_cleaned_grouped["TIME"] = pd.to_datetime(df_cleaned_grouped["DATE_TIME"]).dt.time
df_cleaned_grouped["DATE"] = pd.to_datetime(df_cleaned_grouped["DATE_TIME"]).dt.date
df_cleaned_grouped

,DATE_TIME,DC_POWER,TIME,DATE
0,2020-05-15 00:00:00,0.0,00:00:00,2020-05-15
1,2020-05-15 00:15:00,0.0,00:15:00,2020-05-15
2,2020-05-15 00:30:00,0.0,00:30:00,2020-05-15
3,2020-05-15 00:45:00,0.0,00:45:00,2020-05-15
4,2020-05-15 01:00:00,0.0,01:00:00,2020-05-15
...,...,...,...,...
3254,2020-06-17 22:45:00,0.0,22:45:00,2020-06-17
3255,2020-06-17 23:00:00,0.0,23:00:00,2020-06-17
3256,2020-06-17 23:15:00,0.0,23:15:00,2020-06-17
3257,2020-06-17 23:30:00,0.0,23:30:00,2020-06-17


In [11]:
df_cleaned_grouped.drop(columns=["DATE_TIME"],inplace=True)
df_cleaned_grouped

,DC_POWER,TIME,DATE
0,0.0,00:00:00,2020-05-15
1,0.0,00:15:00,2020-05-15
2,0.0,00:30:00,2020-05-15
3,0.0,00:45:00,2020-05-15
4,0.0,01:00:00,2020-05-15
...,...,...,...
3254,0.0,22:45:00,2020-06-17
3255,0.0,23:00:00,2020-06-17
3256,0.0,23:15:00,2020-06-17
3257,0.0,23:30:00,2020-06-17


In [12]:
# lable encode the date column
df_cleaned_grouped["DATE"] = df_cleaned_grouped["DATE"].astype('category').cat.codes
df_cleaned_grouped

,DC_POWER,TIME,DATE
0,0.0,00:00:00,0
1,0.0,00:15:00,0
2,0.0,00:30:00,0
3,0.0,00:45:00,0
4,0.0,01:00:00,0
...,...,...,...
3254,0.0,22:45:00,33
3255,0.0,23:00:00,33
3256,0.0,23:15:00,33
3257,0.0,23:30:00,33


In [17]:
# get the number of unique values in Date column
df_cleaned_grouped["DATE"].nunique()

34

In [26]:
# get the date of date 5
data_for_epi_date=df_cleaned_grouped[df_cleaned_grouped["DATE"] == 5].reset_index(drop=True)
data_for_epi_date

,DC_POWER,TIME,DATE
0,0.0,00:00:00,5
1,0.0,00:15:00,5
2,0.0,00:30:00,5
3,0.0,00:45:00,5
4,0.0,01:00:00,5
...,...,...,...
91,0.0,22:45:00,5
92,0.0,23:00:00,5
93,0.0,23:15:00,5
94,0.0,23:30:00,5


In [35]:
time_in_time = data_for_epi_date.iloc[95]["TIME"]
time_in_time

datetime.time(23, 45)

In [36]:
normalize_time_from_datetime(time_in_time)

0.9895833333333334

In [15]:
# get  the min max and mean of the DC_POWER
min_dc_power = df_cleaned_grouped["DC_POWER"].min()
max_dc_power = df_cleaned_grouped["DC_POWER"].max()
mean_dc_power = df_cleaned_grouped["DC_POWER"].mean()
min_dc_power, max_dc_power, mean_dc_power

(0.0, 26630.506666666668, np.float64(5124.648465229667))

In [16]:
# save the cleaned data to a csv file
df_cleaned_grouped.to_csv("solar_plant_gym_env\envs\Plant_2_Generation_Data_cleaned.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\e'
C:\Users\gwiku\AppData\Local\Temp\ipykernel_128884\1970487825.py:2: SyntaxWarning: invalid escape sequence '\e'
  df_cleaned_grouped.to_csv("solar_plant_gym_env\envs\Plant_2_Generation_Data_cleaned.csv", index=False)


In [ ]:
# get the list of lengths of DC_POWER column value continuasly have 0 s
zero_lengths = []
zero_length = 0
for dc_power in df_cleaned_grouped["DC_POWER"]:
    if dc_power == 0:
        zero_length += 1
    else:
        if zero_length != 0:
            zero_lengths.append(zero_length)
            zero_length = 0
            
zero_lengths

[24,
 44,
 47,
 49,
 45,
 3,
 45,
 45,
 44,
 45,
 45,
 45,
 46,
 44,
 44,
 44,
 45,
 45,
 45,
 47,
 1,
 2,
 44,
 45,
 44,
 44,
 1,
 44,
 44,
 44,
 44,
 2,
 47,
 1,
 45,
 44,
 1,
 44,
 45,
 44,
 1,
 45]

In [ ]:
# get the max , average and min length of 0 s
max_zero_length = max(zero_lengths)
min_zero_length = min(zero_lengths)
average_zero_length = sum(zero_lengths)/len(zero_lengths)
max_zero_length, min_zero_length, average_zero_length

(49, 1, 36.11904761904762)

In [ ]:
# multiply each max, min and average by 15 minutes to get the time in minutes and devide by 60 to get the time in hours
max_zero_length_hours = max_zero_length*15/60
min_zero_length_hours = min_zero_length*15/60
average_zero_length_hours = average_zero_length*15/60
max_zero_length_hours, min_zero_length_hours, average_zero_length_hours

(12.25, 0.25, 9.029761904761905)

# Final Suggestions:

- ### Constant Output Power:

  `5,000 kW`
  (A balanced, stable value near the average solar input).

- ### Battery Capacity:

  `86 MWh`
  (Enough to handle nighttime and cloudy conditions, with a safety margin).

- ### Battery Charge/Discharge Rate:
  ~`22 MW`
  (To handle the maximum power discrepancy efficiently).
